In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os
from skimage.feature import local_binary_pattern, hog
from sklearn.ensemble import RandomForestClassifier

## Load & Preprocessing Image

In [ ]:
DATASET_PATH = '/content/drive/My Drive/dataset-coral-baru'

In [ ]:
IMAGE_SIZE = (224, 224)
RADIUS = 1
N_POINTS = 8 * RADIUS

images = []
class_list = []

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

for class_index, class_name in enumerate(os.listdir(DATASET_PATH)):
    class_path = os.path.join(DATASET_PATH, class_name)
    if os.path.isdir(class_path):
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            image = cv2.imread(file_path)
            if image is not None:

                image = cv2.resize(image, IMAGE_SIZE)

                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                l = clahe.apply(l)
                lab = cv2.merge((l, a, b))
                image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)


                images.append(image)
                class_list.append(class_index)

images = np.array(images, dtype=np.float32)
class_list = np.array(class_list)

print(f"Total images loaded: {len(images)}")
print(f"Total classes: {len(np.unique(class_list))}")

Total images loaded: 1218
Total classes: 11


In [ ]:
#VGG
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np

vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

model_vgg = Model(inputs=vgg_model.input, outputs=vgg_model.output)

images_vgg = preprocess_input(images)

features_vgg = model_vgg.predict(images_vgg)

print(f"Extracted features shape (VGG16): {features_vgg.shape}")


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step
Extracted features shape (VGG16): (1218, 7, 7, 512)


In [ ]:
#RESNET
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model

resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

model_resnet = Model(inputs=resnet_model.input, outputs=resnet_model.output)

images_resnet = preprocess_input(images)

features_resnet = model_resnet.predict(images_resnet)

print(f"Extracted features shape (ResNet50): {features_resnet.shape}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 107ms/step
Extracted features shape (ResNet50): (1218, 7, 7, 2048)


In [ ]:
#Efficientnet
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model

efficientnet_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

model_efficientnet = Model(inputs=efficientnet_model.input, outputs=efficientnet_model.output)

images_efficientnet = preprocess_input(images)

features_efficientnet = model_efficientnet.predict(images_efficientnet)

print(f"Extracted features shape (EfficientNet): {features_efficientnet.shape}")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 15s 194ms/step
Extracted features shape (EfficientNet): (1218, 7, 7, 1280)


In [ ]:
features_vgg_flattened = features_vgg.reshape(features_vgg.shape[0], -1)
features_resnet_flattened = features_resnet.reshape(features_resnet.shape[0], -1)
features_efficientnet_flattened = features_efficientnet.reshape(features_efficientnet.shape[0], -1)

print(f"Flattened features shape (VGG16): {features_vgg_flattened.shape}")
print(f"Flattened features shape (ResNet50): {features_resnet_flattened.shape}")
print(f"Flattened features shape (EfficientNet): {features_efficientnet_flattened.shape}")


Flattened features shape (VGG16): (1218, 25088)
Flattened features shape (ResNet50): (1218, 100352)
Flattened features shape (EfficientNet): (1218, 62720)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


X_train, X_test, y_train, y_test = train_test_split(features_vgg_flattened, class_list, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(features_resnet_flattened, class_list, test_size=0.2, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(features_efficientnet_flattened, class_list, test_size=0.2, random_state=42)


rf_model = RandomForestClassifier(n_estimators=901,min_samples_split=2,min_samples_leaf=1, random_state=0)
svm_model = SVC(kernel='poly',degree=3,random_state=0)


In [ ]:
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
print(f"SVM (VGG16): {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred))

SVM (VGG16): 42.62%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.69      0.35      0.46        26
           2       1.00      0.53      0.69        17
           3       1.00      0.19      0.32        32
           4       0.77      0.38      0.51        26
           5       0.67      0.21      0.32        19
           6       0.80      0.67      0.73         6
           7       0.94      0.70      0.80        23
           8       0.69      0.36      0.47        25
           9       0.89      0.57      0.69        30
          10       0.14      1.00      0.25        20

    accuracy                           0.43       244
   macro avg       0.69      0.45      0.48       244
weighted avg       0.71      0.43      0.47       244



In [ ]:
svm_model.fit(X_train2, y_train2)

# Predict and evaluate
y_pred2 = svm_model.predict(X_test2)
print(f"SVM (RESNET): {accuracy_score(y_test2, y_pred2) * 100:.2f}%")
print(classification_report(y_test2, y_pred2))

SVM (RESNET): 74.18%
              precision    recall  f1-score   support

           0       0.54      0.35      0.42        20
           1       0.80      0.77      0.78        26
           2       1.00      0.76      0.87        17
           3       1.00      0.53      0.69        32
           4       0.68      0.81      0.74        26
           5       0.77      0.53      0.62        19
           6       1.00      0.67      0.80         6
           7       0.84      0.70      0.76        23
           8       0.71      0.96      0.81        25
           9       0.56      1.00      0.71        30
          10       0.90      0.95      0.93        20

    accuracy                           0.74       244
   macro avg       0.80      0.73      0.74       244
weighted avg       0.78      0.74      0.74       244



In [ ]:
svm_model.fit(X_train3, y_train3)

# Predict and evaluate
y_pred3 = svm_model.predict(X_test3)
print(f"SVM (EFFICIENT NET): {accuracy_score(y_test3, y_pred3) * 100:.2f}%")
print(classification_report(y_test3, y_pred3))

SVM (EFFICIENT NET): 45.08%
              precision    recall  f1-score   support

           0       0.67      0.10      0.17        20
           1       0.61      0.42      0.50        26
           2       1.00      0.24      0.38        17
           3       1.00      0.19      0.32        32
           4       0.83      0.38      0.53        26
           5       0.75      0.16      0.26        19
           6       1.00      0.33      0.50         6
           7       1.00      0.52      0.69        23
           8       1.00      0.60      0.75        25
           9       0.20      1.00      0.33        30
          10       1.00      0.75      0.86        20

    accuracy                           0.45       244
   macro avg       0.82      0.43      0.48       244
weighted avg       0.80      0.45      0.48       244

